# Phosphate accumulation in carbonate-rich brines

In this tutorial, we demonstrate how Reaktoro can be used to perform a series of experiments presented in the paper
of Toner and Catling, 2020 `Toner2020`, which is focused on determining how much phosphate can accumulate by abiotic
processes in carbonate-rich lakes.

```{note}
This tutorial is one of two tutorials that follows
the paper of Toner and Catling `Toner2020` and tries to replicate the geobiological simulations carried out in it.
The second tutorial is available through the link:
[**Carbonate-rich lakes modelling on the early Earth**](geobiology-streammodel-fixed-fugacity.ipynb)
This work was carried out together with Cara Magnabosco and Laura Murzakhmetov, ETH-Zurich.
```

The database is loaded from the database provided by the publication of Toner and Catling (2020):

In [ ]:
from reaktoro import *
import numpy as np
import math

db = PhreeqcDatabase.fromFile('phreeqc-toner-catling.dat') # if running from tutorials folder

We define a chemical system based on the database and provided aqueous and mineral phases. Moreover, to evaluate pH and
phosphate amount in the aqueous phase, we will need aqueous and chemical properties:

In [ ]:
# Define aqueous phase
solution = AqueousPhase(speciate("H O C Na Cl P"))
solution.setActivityModel(chain(
    ActivityModelHKF(),
    ActivityModelDrummond("CO2")
))

# Define mineral phases
minerals = MineralPhases("Natron Nahcolite Trona Na2CO3:H2O Na2CO3:7H2O Halite Na2(HPO4):7H2O")

# Define chemical system
system = ChemicalSystem(db, solution, minerals)

# Define aqueous and chemical properties
props = ChemicalProps(system)
aprops = AqueousProps(system)

To communicate to the solver that fugacity will be constrained in this chemical system, we have to define equilibrium
specifications and corresponding conditions. The first one indicates what will be a constraint and the second one by
which value (set below for the range of fugacities). We also reset the equilibrium option's field `epsilon` to "relax"
the convergence of equilibrium calculations.

In [ ]:
# Define equilibrium specifications
specs = EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.fugacity("CO2")

# Define equilibrium conditions
conditions = EquilibriumConditions(specs)

opts = EquilibriumOptions()
opts.epsilon = 1e-13

For the numerical experiment, we consider Na–Cl-HCO<sub>3</sub>–CO<sub>3</sub>-HPO<sub>4</sub> brine defined
by the initial chemical state in the function `equilibrate_with_NaCl_HPO4().

In [ ]:
def equilibrate_with_NaCl_HPO4(log10pCO2, T):

    conditions.temperature(T, "celsius")
    conditions.pressure(1.0, "atm")
    conditions.fugacity("CO2", 10 ** log10pCO2, "bar")

    state = ChemicalState(system)
    state.set("H2O"           ,   1.0, "kg")
    state.set("Nahcolite"     ,  10.0, "mol")
    state.set("Halite"        ,  10.0, "mol")
    state.set("Na2(HPO4):7H2O",  10.0, "mol")
    state.set("CO2"           , 100.0, "mol")

    solver = EquilibriumSolver(specs)
    solver.setOptions(opts)

    res = solver.solve(state, conditions)

    if not res.optima.succeeded:
        print(f"The optimization solver hasn't converged for T = {T} C and log10pCO2 = {log10pCO2}")
        return math.nan, math.nan, math.nan, math.nan, math.nan

    props.update(state)
    aprops.update(state)

    ph = aprops.pH()[0]
    mCO3 = state.speciesAmount("CO3-2")[0]
    mHCO3 = state.speciesAmount("HCO3-")[0]
    x = 100 * 2 * mCO3 / (mHCO3 + 2 * mCO3)
    mP = props.elementAmountInPhase("P", "AqueousPhase")[0]

    return ph, mCO3, mHCO3, x, mP

To determine the maximum phosphate concentrations possible in such brine, we model solutions saturated with respect to
sodium phosphate, carbonate, and chloride salts at temperatures in a range between 0 to 50 °C and gas pressure
represented log<sub>10</sub>(pCO<sub>2</sub>) = −3.5 to 0 bars. We find that up to 2.1 moles phosphate occurs in
equilibrium with Na<sub>2</sub>(HPO<sub>4</sub>)·7H<sub>2</sub>O salts.

The block below defines the array of the partial CO<sub>2</sub> pressures and data blocks that will be storing results
for different temperatures. We run equilibrium calculations for different pressures in the for-loop:

In [ ]:
num_log10pCO2s = 71
co2pressures = np.flip(np.linspace(-5.0, 2.0, num=num_log10pCO2s))

data_size = 5
data50 = np.zeros((num_log10pCO2s, data_size+1))
data25 = np.zeros((num_log10pCO2s, data_size+1))
data0 = np.zeros((num_log10pCO2s, data_size+1))

for i in range(0, num_log10pCO2s):
    result = equilibrate_with_NaCl_HPO4(co2pressures[i], 0)
    data0[i, 0] = co2pressures[i]
    data0[i, 1] = result[0]
    data0[i, 2] = result[1]
    data0[i, 3] = result[2]
    data0[i, 4] = result[3]
    data0[i, 5] = result[4]

    result = equilibrate_with_NaCl_HPO4(co2pressures[i], 25)
    data25[i, 0] = co2pressures[i]
    data25[i, 1] = result[0]
    data25[i, 2] = result[1]
    data25[i, 3] = result[2]
    data25[i, 4] = result[3]
    data25[i, 5] = result[4]

    result = equilibrate_with_NaCl_HPO4(co2pressures[i], 50)
    data50[i, 0] = co2pressures[i]
    data50[i, 1] = result[0]
    data50[i, 2] = result[1]
    data50[i, 3] = result[2]
    data50[i, 4] = result[3]
    data50[i, 5] = result[4]

The modeled pH of saturated phosphate brines depends on the temperature and the partial CO<sub>2</sub> pressures.
At present day pCO<sub>2</sub> levels (log<sub>10</sub>(pCO<sub>2</sub>) = −3.5), solutions are highly alkaline
(pH approximate to 10), consistent with high pHs measured in modern soda lakes. However, in CO<sub>2</sub>-rich
atmospheres on the early Earth (log<sub>10</sub>(pCO<sub>2</sub>) = −2 to 0), brines range from moderately alkaline
(with pH = 9) to slightly acidic (pH = 6.5) because of acidification by CO<sub>2</sub> (see the plot below).
We note that plotted pH levels are the maximum values for the corresponding solution, as it is saturated with respect to
carbonate minerals. For undersaturated solutions, the pH is always lower. We see that temperature also affects the pH
levels, because CO<sub>2</sub> is more soluble in solutions at lower temperatures, making pH slightly higher.

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(data0[:, 0], data0[:, 1], label=f'0 C', color='C2')
plt.plot(data0[:, 0], data25[:, 1], label=f'25 C', color='C3')
plt.plot(data0[:, 0], data50[:, 1], label=f'50 C', color='C4')
plt.legend(loc="best")
plt.xlabel(r'$\log_{10}(\rm{pCO}_2)$ [bar]')
plt.ylabel('pH [-]')
plt.grid()

Below, we see that the solubility of phosphate increases with growing temperature and pressure:

In [ ]:
plt.figure()
plt.title("Solubility of phosphate")
plt.plot(co2pressures, data50[:, 5], label=f'50 C', color="C2")
plt.plot(co2pressures, data25[:, 5], label=f'25 C', color="C3")
plt.plot(co2pressures, data0[:, 5], label=f'0 C', color="C4")
plt.legend(loc="best")
plt.xlabel(r'$\log_{10}(\sf{pCO}_2)$ [bar]')
plt.ylabel('Amount of P [mol]')
plt.grid()

The relative proportion of CO<sub>2</sub><sup>−3</sup> vs. HCO<sup>−3</sup> ions in solution also controls the
log<sub>10</sub>(pCO<sub>2</sub>) and pH modeled for the experimental saturated Na–HCO<sub>3</sub>–CO<sub>3</sub> brines.
Here, the pH ranges from neutral (at 1 bar CO<sub>2</sub>) to pH = 9 at 0.01 bar partial CO<sub>2</sub> pressure.
The relatively high pCO<sub>2</sub> values would acidify the solution. This suggests that increased phosphate
concentrations could have occurred in CO<sub>2</sub>-rich atmospheres on the early Earth. The x-axis corresponds to the
equivalent percentage of CO<sub>2</sub><sup>−3</sup> ions relative to the total carbonate alkalinity defined as and is
defined as $x = \frac{2[\sf{CO}_3^{-2}]}{[\sf{HCO}_3^-] + 2[\sf{CO}_3^{-2}]}$.

In [ ]:
fig, ax1 = plt.subplots()
color = 'tab:red'
ax1.set_xlabel(r'$\frac{2[\sf{CO}_3^{-2}]}{[\sf{HCO}_3^-] + 2[\sf{CO}_3^{-2}]}$ [%]')
ax1.set_ylabel('pH [-]', color=color)
ax1.plot(data25[:, 4], data25[:, 1], color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
color = 'tab:blue'
ax2.set_ylabel(r'$\log_{10}(\sf{pCO}_2)$ [bar]', color=color)  # we already handled the x-label with ax1
ax2.plot(data25[:, 4], data25[:, 0], color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.grid()